## Demo 
**Dataset** : Multiple target variable classification- Hackathon
    
    -Context
    This data is taken from a Machine Learning Hackathon, in which i participated recently

    -Content
    The dataset consist of Physical attributes of any pet such as condition, colour, length, height, issue_date, listing_date, X1 & X2 etc.

    -Inspiration
    I have participated in this competition, but couldnt get a accuracy score of more than 90.82. So, i am keeping it in this open forum to participate & helps us identify the best approach / method to be used for this problem


Infomacion dataset en el siguiente enlace: [link](https://www.kaggle.com/datasets/ppsheth91/two-target-variables-classification-problem?select=train.csv)
    
    

In [1]:
!pip install -q awswrangler

In [12]:
import awswrangler as wr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rcParams["figure.figsize"] =8,4 # para ajustar el tamaño de imagen
import warnings
warnings.filterwarnings("ignore") #para evitar errores de operaciones matematicas
import random
import statistics
import math
#from pandas_profiling import ProfileReport

In [13]:
dataset = wr.s3.read_csv(path='s3://data-tecnicas-muestreo/datasets/train_multiple.csv')

In [14]:
dataset.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,3
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [15]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18834 entries, 0 to 18833
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pet_id          18834 non-null  object 
 1   issue_date      18834 non-null  object 
 2   listing_date    18834 non-null  object 
 3   condition       17357 non-null  float64
 4   color_type      18834 non-null  object 
 5   length(m)       18834 non-null  float64
 6   height(cm)      18834 non-null  float64
 7   X1              18834 non-null  int64  
 8   X2              18834 non-null  int64  
 9   breed_category  18834 non-null  float64
 10  pet_category    18834 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 1.6+ MB


In [5]:
#dataset.rename(columns = {'Loan Status':'Loan_Status'}, inplace = True)
#dataset.rename(columns = {'Total Collection Amount':'total'}, inplace = True)  

In [72]:
#wr.s3.to_csv(df = dataset, path = 's3://data-tecnicas-muestreo/datasets/train_multiple.csv', index=False)

In [6]:
#dataset['pet_category'].value_counts().sort_index().index.tolist()

In [7]:
#dataset['pet_category'].value_counts()/len(target)

In [75]:
# dataset.rename(columns = {'Loan Status':'Loan_Status'}, inplace = True)
# dataset.rename(columns = {'Total Collection Amount':'total'}, inplace = True)  

In [17]:
#dataset['pet_category'] = dataset['pet_category'].replace(4, 3)

In [18]:
dataset.pet_category.value_counts()

2    10621
1     7184
3      941
0       88
Name: pet_category, dtype: int64

In [19]:
dataset.X1.describe()

count    18834.000000
mean         5.369598
std          6.572366
min          0.000000
25%          0.000000
50%          0.000000
75%         13.000000
max         19.000000
Name: X1, dtype: float64

In [20]:
wr.s3.to_csv(df = dataset, path = 's3://data-tecnicas-muestreo/datasets/train_multiple.csv', index=False)

{'paths': ['s3://data-tecnicas-muestreo/datasets/train_multiple.csv'],
 'partitions_values': {}}

### Definimos la variable target y la variable a estimar

In [21]:
target = dataset.pet_category
estimador = dataset.X1

In [23]:
def estratificado (target, estimador, LEE):
    v = []
    p = []
    n = []
    L = []
    ni = []
    index_sample = []
    mean_est = []
    target_sample = []
    var_mean_est = []
    error_est = []
    LI = []
    LS = []
    data_filtrada = pd.DataFrame()
    
    
    
    N = len(dataset)
   
    D =  pow(LEE, 2)/4


    
    for i in target.value_counts().sort_index().index.tolist(): # range(0, len(target.unique()))
        
        v.append(statistics.variance(dataset[target == sorted(target.unique())[i]][estimador.name]))
        p.append(target.value_counts()[i]/len(target))
        n.append(target.value_counts()[i])
        
        L = (np.multiply(np.power(n, 2), v)/p)/( np.power(N, 2)*D + np.multiply(n, v)   )
        
        
        
    for i in target.value_counts().sort_index().index.tolist():   
        
        ni = round((round(np.sum(L))* target.value_counts()/len(target)))
        
        
        index_sample.append(random.sample(list(dataset[target == sorted(target.unique())[i]].index), int(ni[i]) )) # 
        
        target_sample.append(estimador[index_sample[i]])
    
        mean_est.append(target_sample[i].mean())
        
        var_mean_est.append((np.var(target_sample[i])/ni[i])*(1-(ni[i]/n[i])))  # varianza proporcion CP estimada muestra
        
        error_est.append(2*math.sqrt(var_mean_est[i]))
        
        LI.append(mean_est[i] - error_est[i])  # limite superior estimado estimacion poblacional
        LS.append(mean_est[i] + error_est[i])  # limite inferior estimado estimacion poblacional
        
        data_filtrada = data_filtrada.append(dataset.iloc[target_sample[i].index, :])
        
        print("Estrato o categoria:", i)
        print("El intervalo de confianza de la proporcion esta entre", "[",LI[i], LS[i],"]")
        print("tamaño total poblacional:" , n[i] )
        print("tamaño de muestra estimada:" , round(ni[i] ))
        print("promedio poblacional estimado:" , dataset[target == sorted(target.unique())[i]][estimador.name].mean() )
        print("promedio muestral estimado:" , mean_est[i] )
        print("error de estimacion:" , error_est[i] )
        print("")
        
        
    print("distribucion en porcentaje variable target (poblacion)")   
    print(target.value_counts().sort_index()*100/len(dataset))
    print("")
    print("distribucion en porcentaje variable target (muestra)")
    print(data_filtrada[target.name].value_counts().sort_index()*100/len(data_filtrada))
    
        
    return(data_filtrada)

In [24]:
df1 = estratificado(target, estimador, 0.2)

Estrato o categoria: 0
El intervalo de confianza de la proporcion esta entre [ 6.14506775540828 12.72993224459172 ]
tamaño total poblacional: 88
tamaño de muestra estimada: 16
promedio poblacional estimado: 6.5227272727272725
promedio muestral estimado: 9.4375
error de estimacion: 3.29243224459172

Estrato o categoria: 1
El intervalo de confianza de la proporcion esta entre [ 7.424996316612049 8.099632299494285 ]
tamaño total poblacional: 7184
tamaño de muestra estimada: 1279
promedio poblacional estimado: 7.56055122494432
promedio muestral estimado: 7.762314308053167
error de estimacion: 0.33731799144111796

Estrato o categoria: 2
El intervalo de confianza de la proporcion esta entre [ 3.1963730773021792 3.6645470707676253 ]
tamaño total poblacional: 10621
tamaño de muestra estimada: 1891
promedio poblacional estimado: 3.263911119480275
promedio muestral estimado: 3.4304600740349023
error de estimacion: 0.234086996732723

Estrato o categoria: 3
El intervalo de confianza de la proporci

In [25]:
df1.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
8468,ANSL_57882,2017-01-09 00:00:00,2018-02-03 13:11:00,NaN,Gray,0.86,35.39,0,7,2.0,0
1675,ANSL_58939,2017-05-14 00:00:00,2017-12-10 13:57:00,NaN,Brown,0.83,20.29,0,1,2.0,0
8349,ANSL_66880,2015-08-04 00:00:00,2016-08-25 13:13:00,NaN,Brown,0.52,11.01,18,4,2.0,0
18645,ANSL_76740,2018-04-01 00:00:00,2019-05-09 19:42:00,2.0,Black,0.52,30.31,7,1,1.0,0
14619,ANSL_73298,2016-01-10 00:00:00,2017-01-31 19:28:00,NaN,Orange,0.76,43.74,18,4,2.0,0


### Interpretacion



El objetivo final del muestreo estratificado es mantener la proporcionalidad de las categoria en interes, en este caso los diferentes tipo de mascota, asi comparamos la tabla de distribucion de la poblacion vs la tabla de distribucion de la muestra, se observa que se asemejan de proporcional las diferentes distribucion de la categoria de interes, esto es lo que buscamos.

Cada estrato (pet_category)  se intepreta tal cual un muestreo aleatorio simple.

Importante señalar que para cada estrato se ejecuta un diferente error de estimacion, esto es por los distintas cantidades de elemento en cada estrato.



